In [ ]:
experiment_name = 'Clip16_NIGHTS_Lora_Default_123_First'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import logging
import os
import sys
import yaml
import wandb

# Project setup
PROJECT_ROOT = '/content/drive/MyDrive/perceptual-vits-fashion-forecasting'
sys.path.append(os.path.join(PROJECT_ROOT, 'src'))

# Load config
config_path = os.path.join(PROJECT_ROOT, 'configs', f"{experiment_name}.yaml")
if not os.path.isfile(config_path):
    raise FileNotFoundError(f"Config file not found: {config_path}")

with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')

Mounted at /content/drive


In [ ]:
# Log into WnadB
with open(os.path.join(PROJECT_ROOT, 'wandb_key.txt')) as file:
    key = file.read().strip()

wandb.login(key=key)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nils-grossepieper (nils-grossepieper-eberhard-karls-universit-t-t-bingen) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Log the config files as artefact in WandB
with wandb.init(
    project=config['global']['project_name'],
    name=f"{config['global']['experiment_name']}_experiment_parameters",
    group=config['global']['experiment_name'],
    job_type='Prediction experiment parameters',
    tags=[config['global']['project_name'], 'configs', 'prediction'],
    notes=f"YAML config for {config['global']['experiment_name']} experiment."
) as run:
    art = wandb.Artifact(
        name=f"{config['global']['experiment_name']}_config",
        type="config",
        description='Configuration file for prediction model training'
    )
    art.add_file(config_path, name="config.yaml")
    run.log_artifact(art)

In [ ]:
# Inspect GPU
!nvidia-smi

# (Optional) Remove conflicting installs from previous sessions
!pip -q uninstall -y tensorflow tensorflow-gpu keras jax jaxlib \
  torch torchvision torchaudio nvidia-cudnn-cu12 nvidia-cublas-cu12 pillow || true

# Install a CUDA-matched PyTorch FIRST (Colab usually uses CUDA 12.1) ---
!pip -q install --index-url https://download.pytorch.org/whl/cu121 \
  torch torchvision torchaudio

# Install DreamSim deps WITHOUT pulling another torch build
# (Everything here excludes torch/vision/torchaudio)
!pip -q install \
  configargparse dists-pytorch lpips numpy pandas pytorch-lightning \
  PyYAML scipy tensorboard timm torchmetrics tqdm transformers \
  open-clip-torch peft>=0.2.0

# Pillow upgrade (fix ImportError: is_directory)
!pip -q install 'pillow>=10.1.0'

# OpenAI CLIP (git)
!pip -q install git+https://github.com/openai/CLIP.git

# (Optional) Tame CUDA allocator fragmentation for long sessions
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True,max_split_size_mb:128'

# Install Optuna
!pip -q install optuna
!pip install timm

/bin/bash: line 1: nvidia-smi: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 125.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6

In [ ]:
from data.process_visuelle import process_visuelle

# Process Visuelle2 data
results_process_visuelle = process_visuelle(
    season=config['data']['season'],
    split_method=config['data']['split_method'],
    melt_data=config['global']['melt_data'],
    project_root=config['global']['project_root']
)

[INFO] NumExpr defaulting to 8 threads.
[INFO] Starting Visuelle2 data processing...
[INFO] All processed files already exist. Skipping processing.


In [ ]:
# Load the dreamsim library if needed
if config['vision_model']['load_vit'] and config['vision_model']['dataset_name'] == 'nights':
  !pip install -q dreamsim

from data.create_visuelle_embeddings import create_visuelle_embeddings

# Create Visuelle2 embeddings or skipp if they already exist
embedding_path = create_visuelle_embeddings(
    model_family=config['vision_model']['model_family'],
    train=config['vision_model']['train'],
    load_vit=config['vision_model']['load_vit'],
    model_type=config['vision_model']['model_type'],
    training_method=config['vision_model']['training_method'],
    feat_type=config['vision_model']['vit']['feat_type'],
    stride=config['vision_model']['vit']['stride'],
    hidden_size=config['vision_model']['hidden_size'],
    dataset_name=config['vision_model']['dataset_name'],
    tag=config['vision_model']['tag'],
    best=config['vision_model']['best'],
    project_root=config['global']['project_root']
)

[INFO] Creating embeddings for Clip16_DreamSim_Lora_Second_Model_clip_vitb16_lora_embedding_16_nights
[INFO] No precomputed embeddings found. Creating new embeddings...
[INFO] Using Vision Transformer model
[INFO] Creating backbone vision transformer for clip_vitb16
[INFO] Loading Vision Transformer model trained with LoRA
Embedding images: 100%|██████████| 5355/5355 [1:32:35<00:00,  1.04s/it]
[WARNING] Skipped 1 files. See /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/embeddings/Clip16_DreamSim_Lora_Second_Model_clip_vitb16_lora_embedding_16_nights_bad_images.txt
[INFO] Saved embeddings to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/embeddings/Clip16_DreamSim_Lora_Second_Model_clip_vitb16_lora_embedding_16_nights_embeddings.csv


In [ ]:
import gc
from contextlib import contextmanager
from data.load_visuelle import load_visuelle
from utils_prediction.test_metrics import evaluate_weekly_metrics

# Context manager to remove large files in memory
@contextmanager
def auto_release_large_objects(obj_name_list):
    """Automatically deletes large objects from memory after use."""
    try:
        yield
    finally:
        for obj_name in obj_name_list:
            if obj_name in globals():
                del globals()[obj_name]
        gc.collect()

# Train and evaluate kNN regressor
if config['forecasting']['knn']['enabled']:

    with auto_release_large_objects([
        'dict_load_visuelle_knn',
        'results_train_knn_regressor',
        'results_test_knn_regressor'
    ]):
        # Load dataset with embeddings
        dict_load_visuelle_knn = load_visuelle(
            process_data_dict=results_process_visuelle,
            embedding_path=embedding_path,
            using_year_int=config['features']['using_year_int'],
            using_year_dummies=config['features']['using_year_dummies'],
            using_season_dummies=config['features']['using_season_dummies'],
            using_price_float=config['features']['using_price_float'],
            using_category_dummies=config['features']['using_category_dummies'],
            using_color_dummies=config['features']['using_color_dummies'],
            using_fabric_dummies=config['features']['using_fabric_dummies'],
            using_store_int=config['features']['using_store_int'],
            using_store_dummies=config['features']['using_store_dummies'],
            using_week_dummies=config['features']['using_week_dummies'],
            pca=config['features']['pca'],
            n_components=config['features']['n_components'],
            visualize_pca=config['features']['visualize_pca'],
            melt_data=config['global']['melt_data'],
            dummy_normalization=config['forecasting']['knn']['dummy_normalization'],
            normalize_embeddings_manually=config['forecasting']['knn']['normalize_embeddings_manually'],
            project_root=config['global']['project_root']
        )

        from training.train_knn_regressor import train_knn_regressor

        # Train model
        results_train_knn_regressor = train_knn_regressor(
            dict_data=dict_load_visuelle_knn,
            project_name=config['global']['project_name'],
            experiment_name=config['global']['experiment_name'],
            wandb_bool=config['global']['wandb_bool'],
            error_metric=config['forecasting']['knn']['error_metric'],
            max_neighbors=config['forecasting']['knn']['max_neighbors'],
            weights=config['forecasting']['knn']['weights'],
            pca_dim=dict_load_visuelle_knn['scalars']['pca_dim'],
            project_root=config['global']['project_root']
        )

        # Evaluate model
        results_test_knn_regressor = evaluate_weekly_metrics(
            dict_model=results_train_knn_regressor,
            dict_data=dict_load_visuelle_knn,
            project_name=config['global']['project_name'],
            experiment_name=config['global']['experiment_name'],
            melt_data=config['global']['melt_data'],
            wandb_bool=config['global']['wandb_bool'],
            project_root=config['global']['project_root'],
            scalar=results_process_visuelle['scalar']
        )

else:
    print('Skip kNN training.')

[INFO] Loading and preparing Visuelle2 dataset...
[INFO] Load training data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_train.csv
[INFO] Load validation data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_val.csv
[INFO] Load test data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_test.csv
[INFO] Load embeddings from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/embeddings/Clip16_DreamSim_Lora_Second_Model_clip_vitb16_lora_embedding_16_nights_embeddings.csv
[INFO] Length of training data before merging is 51444 rows.
[INFO] Length of training data after merging is 51432 rows.
[INFO] Length of validation data before merging is 6408 rows.
[INFO] Length of validation data after merging is 

  0%|          | 0/400 [00:00<?, ?it/s]

[INFO] Initializing KNN regressor with: n_neighbors=67, weights='uniform', final_run=False
[INFO] Initializing KNN regressor with: n_neighbors=155, weights='distance', final_run=False
[INFO] Initializing KNN regressor with: n_neighbors=171, weights='distance', final_run=False
[INFO] Initializing KNN regressor with: n_neighbors=99, weights='uniform', final_run=False
[INFO] Initializing KNN regressor with: n_neighbors=124, weights='uniform', final_run=False
[INFO] Initializing KNN regressor with: n_neighbors=31, weights='uniform', final_run=False
[INFO] Initializing KNN regressor with: n_neighbors=78, weights='distance', final_run=False
[INFO] Initializing KNN regressor with: n_neighbors=131, weights='distance', final_run=False
[INFO] Initializing KNN regressor with: n_neighbors=71, weights='distance', final_run=False


[I 2025-10-31 11:11:47,289] Trial 0 finished with value: 0.3895855062232078 and parameters: {'n_neighbors': 67, 'weights': 'uniform'}. Best is trial 0 with value: 0.3895855062232078.


[INFO] Initializing KNN regressor with: n_neighbors=108, weights='uniform', final_run=False


...

[INFO] Hyperparameter search took 1141.29 seconds.
[INFO] Best parameters: study.best_params['n_neighbors']=31 study.best_params['weights']='distance'
[INFO] Best MSE: 0.334513315827412
[INFO] Starting final model training with best parameters: study.best_params['n_neighbors']=31 study.best_params['weights']='distance'
[INFO] Initializing KNN regressor with: n_neighbors=31, weights='distance', final_run=True
[INFO] Final training took 0.05 seconds.
[INFO] Saving final kNN model to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/models/Clip16_DreamSim_Lora_Default_123_Third/knn_regressor_model.joblib and study to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/studies/Clip16_DreamSim_Lora_Default_123_Third/knn_regressor_study.joblib.


[I 2025-10-31 11:30:03,898] Trial 399 finished with value: 0.4103660962229647 and parameters: {'n_neighbors': 87, 'weights': 'uniform'}. Best is trial 169 with value: 0.334513315827412.


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[INFO] Evaluating metrics for model: /content/drive/MyDrive/perceptual-vits-fashion-forecasting/models/Clip16_DreamSim_Lora_Default_123_Third/knn_regressor_model.joblib
[INFO] Results saved: /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/results_knn_regressor_model.csv, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/avg_results_knn_regressor_model.json, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/global_results_knn_regressor_model.json, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/forecasts/Clip16_DreamSim_Lora_Default_123_Third/forecasts_knn_regressor_model.npy


knn_regressor_model_avg/mae,▁
knn_regressor_model_avg/mape_pct,▁
knn_regressor_model_avg/mse,▁
knn_regressor_model_avg/r2,▁
knn_regressor_model_avg/residual_std,▁
knn_regressor_model_avg/rmse,▁
knn_regressor_model_avg/standard_std,▁
knn_regressor_model_avg/ts,▁
knn_regressor_model_avg/wape_pct,▁
knn_regressor_model_global/mae,▁
+18,...


In [ ]:
# Train and evaluate ridge regressor
if config['forecasting']['ridge']['enabled']:

    with auto_release_large_objects([
        'dict_load_visuelle_ridge',
        'results_train_ridge_regressor',
        'results_test_ridge_regressor'
    ]):
        # Load dataset with embeddings
        dict_load_visuelle_ridge = load_visuelle(
            process_data_dict=results_process_visuelle,
            embedding_path=embedding_path,
            using_year_int=config['features']['using_year_int'],
            using_year_dummies=config['features']['using_year_dummies'],
            using_season_dummies=config['features']['using_season_dummies'],
            using_price_float=config['features']['using_price_float'],
            using_category_dummies=config['features']['using_category_dummies'],
            using_color_dummies=config['features']['using_color_dummies'],
            using_fabric_dummies=config['features']['using_fabric_dummies'],
            using_store_int=config['features']['using_store_int'],
            using_store_dummies=config['features']['using_store_dummies'],
            using_week_dummies=config['features']['using_week_dummies'],
            pca=config['features']['pca'],
            n_components=config['features']['n_components'],
            visualize_pca=config['features']['visualize_pca'],
            melt_data=config['global']['melt_data'],
            dummy_normalization=config['forecasting']['ridge']['dummy_normalization'],
            normalize_embeddings_manually=config['forecasting']['ridge']['normalize_embeddings_manually'],
            project_root=config['global']['project_root']
        )

        from training.train_ridge_regressor import train_ridge_regressor

        # Train model
        results_train_ridge_regressor = train_ridge_regressor(
            dict_data=dict_load_visuelle_ridge,
            project_name=config['global']['project_name'],
            experiment_name=config['global']['experiment_name'],
            wandb_bool=config['global']['wandb_bool'],
            error_metric=config['forecasting']['ridge']['error_metric'],
            alpha_values=config['forecasting']['ridge']['alpha_values'],
            melt_data=config['global']['melt_data'],
            pca_dim=dict_load_visuelle_ridge['scalars']['pca_dim'],
            forecasting_seed=config['global']['forecasting_seed'],
            project_root=config['global']['project_root']
        )

        # Evaluate model
        results_test_ridge_regressor = evaluate_weekly_metrics(
            dict_model=results_train_ridge_regressor,
            dict_data=dict_load_visuelle_ridge,
            project_name=config['global']['project_name'],
            experiment_name=config['global']['experiment_name'],
            melt_data=config['global']['melt_data'],
            wandb_bool=config['global']['wandb_bool'],
            project_root=config['global']['project_root'],
            scalar=results_process_visuelle['scalar']
        )

else:
    print('Skip ridge regression training.')

[INFO] Loading and preparing Visuelle2 dataset...
[INFO] Load training data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_train.csv
[INFO] Load validation data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_val.csv
[INFO] Load test data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_test.csv
[INFO] Load embeddings from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/embeddings/Clip16_DreamSim_Lora_Second_Model_clip_vitb16_lora_embedding_16_nights_embeddings.csv
[INFO] Length of training data before merging is 51444 rows.
[INFO] Length of training data after merging is 51432 rows.
[INFO] Length of validation data before merging is 6408 rows.
[INFO] Length of validation data after merging is 

  0%|          | 0/9 [00:00<?, ?it/s]

[INFO] Testing Ridge regressor with alpha=1000
[INFO] Testing Ridge regressor with alpha=0.01
[INFO] Testing Ridge regressor with alpha=0.001
[INFO] Testing Ridge regressor with alpha=1
[INFO] Initializing Ridge regressor with: alpha=1000, melt_data=True, final_run=Falseforecasting_seed=123
[INFO] Testing Ridge regressor with alpha=10000
[INFO] Testing Ridge regressor with alpha=100
[INFO] Initializing Ridge regressor with: alpha=0.01, melt_data=True, final_run=Falseforecasting_seed=123
[INFO] Testing Ridge regressor with alpha=0.1
[INFO] Initializing Ridge regressor with: alpha=0.001, melt_data=True, final_run=Falseforecasting_seed=123
[INFO] Testing Ridge regressor with alpha=0.0001
[INFO] Initializing Ridge regressor with: alpha=1, melt_data=True, final_run=Falseforecasting_seed=123
[INFO] Initializing Ridge regressor with: alpha=10000, melt_data=True, final_run=Falseforecasting_seed=123
[INFO] Initializing Ridge regressor with: alpha=100, melt_data=True, final_run=Falseforecasting_

[I 2025-10-31 11:30:54,755] Trial 7 finished with value: 0.22293080957413008 and parameters: {'alpha': 0.0001}. Best is trial 7 with value: 0.22293080957413008.
[I 2025-10-31 11:30:54,788] Trial 1 finished with value: 0.22194394893089436 and parameters: {'alpha': 0.01}. Best is trial 1 with value: 0.22194394893089436.
[I 2025-10-31 11:30:54,792] Trial 4 finished with value: 0.7020063909549056 and parameters: {'alpha': 10000}. Best is trial 1 with value: 0.22194394893089436.
[I 2025-10-31 11:30:54,795] Trial 6 finished with value: 0.2159400485284702 and parameters: {'alpha': 0.1}. Best is trial 6 with value: 0.2159400485284702.
[I 2025-10-31 11:30:54,797] Trial 0 finished with value: 0.3974647903754224 and parameters: {'alpha': 1000}. Best is trial 6 with value: 0.2159400485284702.
[I 2025-10-31 11:30:54,798] Trial 2 finished with value: 0.22283719662333512 and parameters: {'alpha': 0.001}. Best is trial 6 with value: 0.2159400485284702.
[I 2025-10-31 11:30:54,800] Trial 5 finished with

[INFO] Hyperparameter search took 12.87 seconds.
[INFO] Best alpha: 10
[INFO] Best MSE: 0.20100080336817505
[INFO] Starting final model training with best alpha: 10...
[INFO] Initializing Ridge regressor with: alpha=10, melt_data=True, final_run=Trueforecasting_seed=123


[I 2025-10-31 11:30:55,165] Trial 8 finished with value: 0.20100080336817505 and parameters: {'alpha': 10}. Best is trial 8 with value: 0.20100080336817505.


[INFO] Final training took 0.21 seconds.
[INFO] Saving final Ridge model to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/models/Clip16_DreamSim_Lora_Default_123_Third/ridge_regressor_model.joblib and study to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/studies/Clip16_DreamSim_Lora_Default_123_Third/ridge_regressor_study.joblib.


[INFO] Evaluating metrics for model: /content/drive/MyDrive/perceptual-vits-fashion-forecasting/models/Clip16_DreamSim_Lora_Default_123_Third/ridge_regressor_model.joblib
[INFO] Results saved: /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/results_ridge_regressor_model.csv, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/avg_results_ridge_regressor_model.json, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/global_results_ridge_regressor_model.json, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/forecasts/Clip16_DreamSim_Lora_Default_123_Third/forecasts_ridge_regressor_model.npy


ridge_regressor_model_avg/mae,▁
ridge_regressor_model_avg/mape_pct,▁
ridge_regressor_model_avg/mse,▁
ridge_regressor_model_avg/r2,▁
ridge_regressor_model_avg/residual_std,▁
ridge_regressor_model_avg/rmse,▁
ridge_regressor_model_avg/standard_std,▁
ridge_regressor_model_avg/ts,▁
ridge_regressor_model_avg/wape_pct,▁
ridge_regressor_model_global/mae,▁
+18,...


In [ ]:
# Train and evaluate random forest regressor
if config['forecasting']['random_forest']['enabled']:

    with auto_release_large_objects([
        'dict_load_visuelle_rf',
        'results_train_rf_regressor',
        'results_test_rf_regressor'
    ]):
        # Load dataset with embeddings
        dict_load_visuelle_rf = load_visuelle(
            process_data_dict=results_process_visuelle,
            embedding_path=embedding_path,
            using_year_int=config['features']['using_year_int'],
            using_year_dummies=config['features']['using_year_dummies'],
            using_season_dummies=config['features']['using_season_dummies'],
            using_price_float=config['features']['using_price_float'],
            using_category_dummies=config['features']['using_category_dummies'],
            using_color_dummies=config['features']['using_color_dummies'],
            using_fabric_dummies=config['features']['using_fabric_dummies'],
            using_store_int=config['features']['using_store_int'],
            using_store_dummies=config['features']['using_store_dummies'],
            using_week_dummies=config['features']['using_week_dummies'],
            pca=config['features']['pca'],
            n_components=config['features']['n_components'],
            visualize_pca=config['features']['visualize_pca'],
            melt_data=config['global']['melt_data'],
            dummy_normalization=config['forecasting']['random_forest']['dummy_normalization'],
            normalize_embeddings_manually=config['forecasting']['random_forest']['normalize_embeddings_manually'],
            project_root=config['global']['project_root']
        )

        from training.train_random_forest_regressor import train_random_forest_regressor

        # Train model
        results_train_rf_regressor = train_random_forest_regressor(
            dict_data=dict_load_visuelle_rf,
            project_name=config['global']['project_name'],
            experiment_name=config['global']['experiment_name'],
            wandb_bool=config['global']['wandb_bool'],
            error_metric=config['forecasting']['random_forest']['error_metric'],
            n_trials=config['forecasting']['random_forest']['n_trials'],
            n_estimators=config['forecasting']['random_forest']['n_estimators'],
            max_depth=config['forecasting']['random_forest']['max_depth'],
            min_samples_split=config['forecasting']['random_forest']['min_samples_split'],
            min_samples_leaf=config['forecasting']['random_forest']['min_samples_leaf'],
            melt_data=config['global']['melt_data'],
            pca_dim=dict_load_visuelle_rf['scalars']['pca_dim'],
            forecasting_seed=config['global']['forecasting_seed'],
            project_root=config['global']['project_root']
        )

        # Evaluate model
        results_test_rf_regressor = evaluate_weekly_metrics(
            dict_model=results_train_rf_regressor,
            dict_data=dict_load_visuelle_rf,
            project_name=config['global']['project_name'],
            experiment_name=config['global']['experiment_name'],
            melt_data=config['global']['melt_data'],
            wandb_bool=config['global']['wandb_bool'],
            project_root=config['global']['project_root'],
            scalar=results_process_visuelle['scalar']
        )

else:
    print('Skip random forest training.')

[INFO] Loading and preparing Visuelle2 dataset...
[INFO] Load training data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_train.csv
[INFO] Load validation data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_val.csv
[INFO] Load test data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_test.csv
[INFO] Load embeddings from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/embeddings/Clip16_DreamSim_Lora_Second_Model_clip_vitb16_lora_embedding_16_nights_embeddings.csv
[INFO] Length of training data before merging is 51444 rows.
[INFO] Length of training data after merging is 51432 rows.
[INFO] Length of validation data before merging is 6408 rows.
[INFO] Length of validation data after merging is 

  0%|          | 0/40 [00:00<?, ?it/s]

[INFO] Initializing Random Forest regressor with: n_estimators=700, max_depth=13, min_samples_split=2, min_samples_leaf=5, melt_data=True, final_run=False
[INFO] Initializing Random Forest regressor with: n_estimators=700, max_depth=13, min_samples_split=2, min_samples_leaf=1, melt_data=True, final_run=False
[INFO] Initializing Random Forest regressor with: n_estimators=700, max_depth=7, min_samples_split=5, min_samples_leaf=1, melt_data=True, final_run=False
[INFO] Initializing Random Forest regressor with: n_estimators=300, max_depth=11, min_samples_split=10, min_samples_leaf=1, melt_data=True, final_run=False
[INFO] Initializing Random Forest regressor with: n_estimators=300, max_depth=11, min_samples_split=5, min_samples_leaf=5, melt_data=True, final_run=False
[INFO] Initializing Random Forest regressor with: n_estimators=500, max_depth=7, min_samples_split=5, min_samples_leaf=5, melt_data=True, final_run=False
[INFO] Initializing Random Forest regressor with: n_estimators=300, max

[I 2025-10-31 11:32:57,301] Trial 3 finished with value: 0.20915689904871032 and parameters: {'n_estimators': 300, 'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 3 with value: 0.20915689904871032.


[INFO] Initializing Random Forest regressor with: n_estimators=500, max_depth=11, min_samples_split=2, min_samples_leaf=1, melt_data=True, final_run=False


...

[INFO] Hyperparameter search took 1160.59 seconds.
[INFO] Best parameters: study.best_params['n_estimators']=700 study.best_params['max_depth']=13, study.best_params['min_samples_split']=2, study.best_params['min_samples_leaf']=1
[INFO] Best MSE: 0.20349335032311397
[INFO] Starting final model training with best parameters: study.best_params['n_estimators']=700 study.best_params['max_depth']=13, study.best_params['min_samples_split']=2, study.best_params['min_samples_leaf']=1
[INFO] Initializing Random Forest regressor with: n_estimators=700, max_depth=13, min_samples_split=2, min_samples_leaf=1, melt_data=True, final_run=True


[I 2025-10-31 11:50:32,671] Trial 39 finished with value: 0.20349335032311397 and parameters: {'n_estimators': 700, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.20349335032311397.


[INFO] Final training took 40.81 seconds.
[INFO] Saving final Random Forest model to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/models/Clip16_DreamSim_Lora_Default_123_Third/random_forest_regressor_model.joblib and study to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/studies/Clip16_DreamSim_Lora_Default_123_Third/random_forest_regressor_study.joblib.


[INFO] Evaluating metrics for model: /content/drive/MyDrive/perceptual-vits-fashion-forecasting/models/Clip16_DreamSim_Lora_Default_123_Third/random_forest_regressor_model.joblib
[INFO] Results saved: /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/results_random_forest_regressor_model.csv, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/avg_results_random_forest_regressor_model.json, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/results/Clip16_DreamSim_Lora_Default_123_Third/global_results_random_forest_regressor_model.json, /content/drive/MyDrive/perceptual-vits-fashion-forecasting/experiments/forecasts/Clip16_DreamSim_Lora_Default_123_Third/forecasts_random_forest_regressor_model.npy


random_forest_regressor_model_avg/mae,▁
random_forest_regressor_model_avg/mape_pct,▁
random_forest_regressor_model_avg/mse,▁
random_forest_regressor_model_avg/r2,▁
random_forest_regressor_model_avg/residual_std,▁
random_forest_regressor_model_avg/rmse,▁
random_forest_regressor_model_avg/standard_std,▁
random_forest_regressor_model_avg/ts,▁
random_forest_regressor_model_avg/wape_pct,▁
random_forest_regressor_model_global/mae,▁
+18,...


In [ ]:
# Train and evaluate gradient boosting regressor
if config['forecasting']['gradient_boosting']['enabled']:

    with auto_release_large_objects([
        'dict_load_visuelle_gb',
        'results_train_gb_regressor',
        'results_test_gb_regressor'
    ]):
        # Load dataset with embeddings
        dict_load_visuelle_gb = load_visuelle(
            process_data_dict=results_process_visuelle,
            embedding_path=embedding_path,
            using_year_int=config['features']['using_year_int'],
            using_year_dummies=config['features']['using_year_dummies'],
            using_season_dummies=config['features']['using_season_dummies'],
            using_category_dummies=config['features']['using_category_dummies'],
            using_price_float=config['features']['using_price_float'],
            using_color_dummies=config['features']['using_color_dummies'],
            using_fabric_dummies=config['features']['using_fabric_dummies'],
            using_store_int=config['features']['using_store_int'],
            using_store_dummies=config['features']['using_store_dummies'],
            using_week_dummies=config['features']['using_week_dummies'],
            pca=config['features']['pca'],
            n_components=config['features']['n_components'],
            visualize_pca=config['features']['visualize_pca'],
            melt_data=config['global']['melt_data'],
            dummy_normalization=config['forecasting']['gradient_boosting']['dummy_normalization'],
            normalize_embeddings_manually=config['forecasting']['gradient_boosting']['normalize_embeddings_manually'],
            project_root=config['global']['project_root']
        )

        from training.train_gradient_boosting_regressor import train_gradient_boosting_regressor

        # Train model
        results_train_gb_regressor = train_gradient_boosting_regressor(
            dict_data=dict_load_visuelle_gb,
            project_name=config['global']['project_name'],
            experiment_name=config['global']['experiment_name'],
            wandb_bool=config['global']['wandb_bool'],
            error_metric=config['forecasting']['gradient_boosting']['error_metric'],
            n_trials=config['forecasting']['gradient_boosting']['n_trials'],
            n_estimators=config['forecasting']['gradient_boosting']['n_estimators'],
            learning_rate=config['forecasting']['gradient_boosting']['learning_rate'],
            max_depth=config['forecasting']['gradient_boosting']['max_depth'],
            min_samples_split=config['forecasting']['gradient_boosting']['min_samples_split'],
            min_samples_leaf=config['forecasting']['gradient_boosting']['min_samples_leaf'],
            melt_data=config['global']['melt_data'],
            pca_dim=dict_load_visuelle_gb['scalars']['pca_dim'],
            forecasting_seed=config['global']['forecasting_seed'],
            project_root=config['global']['project_root']
        )

        # Evaluate model
        results_test_gb_regressor = evaluate_weekly_metrics(
            dict_model=results_train_gb_regressor,
            dict_data=dict_load_visuelle_gb,
            project_name=config['global']['project_name'],
            experiment_name=config['global']['experiment_name'],
            melt_data=config['global']['melt_data'],
            wandb_bool=config['global']['wandb_bool'],
            project_root=config['global']['project_root'],
            scalar=results_process_visuelle['scalar']
        )

else:
    print('Skip gradient boosting training.')

[INFO] Loading and preparing Visuelle2 dataset...
[INFO] Load training data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_train.csv
[INFO] Load validation data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_val.csv
[INFO] Load test data from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/processed_data/melt_true_aw19_default_split/visuelle2_test.csv
[INFO] Load embeddings from /content/drive/MyDrive/perceptual-vits-fashion-forecasting/datasets/visuelle2/embeddings/Clip16_DreamSim_Lora_Second_Model_clip_vitb16_lora_embedding_16_nights_embeddings.csv
[INFO] Length of training data before merging is 51444 rows.
[INFO] Length of training data after merging is 51432 rows.
[INFO] Length of validation data before merging is 6408 rows.
[INFO] Length of validation data after merging is 

  0%|          | 0/40 [00:00<?, ?it/s]

[INFO] Initializing Gradient Boosting Regressor with: n_estimators=500, learning_rate=0.11, max_depth=5, min_samples_split=10, min_samples_leaf=1, melt_data=True, final_run=False, forecasting_seed=123
[INFO] Initializing Gradient Boosting Regressor with: n_estimators=700, learning_rate=0.11, max_depth=7, min_samples_split=10, min_samples_leaf=1, melt_data=True, final_run=False, forecasting_seed=123
[INFO] Initializing Gradient Boosting Regressor with: n_estimators=700, learning_rate=0.09, max_depth=11, min_samples_split=10, min_samples_leaf=3, melt_data=True, final_run=False, forecasting_seed=123
[INFO] Initializing Gradient Boosting Regressor with: n_estimators=500, learning_rate=0.11, max_depth=11, min_samples_split=10, min_samples_leaf=5, melt_data=True, final_run=False, forecasting_seed=123
[INFO] Initializing Gradient Boosting Regressor with: n_estimators=700, learning_rate=0.09, max_depth=9, min_samples_split=5, min_samples_leaf=1, melt_data=True, final_run=False, forecasting_see

[I 2025-10-31 11:53:49,336] Trial 0 finished with value: 0.17781354694891183 and parameters: {'n_estimators': 500, 'learning_rate': 0.11, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.17781354694891183.


[INFO] Initializing Gradient Boosting Regressor with: n_estimators=700, learning_rate=0.05, max_depth=9, min_samples_split=2, min_samples_leaf=5, melt_data=True, final_run=False, forecasting_seed=123


...

[I 2025-10-31 12:05:32,429] Trial 30 finished with value: 0.15989894592325202 and parameters: {'n_estimators': 900, 'learning_rate': 0.07, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 29 with value: 0.15989894592325202.
[I 2025-10-31 12:06:04,484] Trial 31 finished with value: 0.1599097126556509 and parameters: {'n_estimators': 900, 'learning_rate': 0.05, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 29 with value: 0.15989894592325202.
[I 2025-10-31 12:07:41,172] Trial 32 finished with value: 0.1599097126556509 and parameters: {'n_estimators': 900, 'learning_rate': 0.05, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 29 with value: 0.15989894592325202.
[I 2025-10-31 12:07:48,432] Trial 34 finished with value: 0.1599097126556509 and parameters: {'n_estimators': 900, 'learning_rate': 0.05, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 29 with value: 0.1598989459

[INFO] Hyperparameter search took 1008.27 seconds.
[INFO] Best parameters: study.best_params['n_estimators']=900 study.best_params['learning_rate']=0.07, study.best_params['max_depth']=5, study.best_params['min_samples_split']=2, study.best_params['min_samples_leaf']=5
[INFO] Best MSE: 0.15989894592325202
[INFO] Starting final model training with best parameters: study.best_params['n_estimators']=900 study.best_params['learning_rate']=0.07, 5, study.best_params['min_samples_split']=2, study.best_params['min_samples_leaf']=5
[INFO] Initializing Gradient Boosting Regressor with: n_estimators=900, learning_rate=0.07, max_depth=5, min_samples_split=2, min_samples_leaf=5, melt_data=True, final_run=True, forecasting_seed=123


[I 2025-10-31 12:08:39,647] Trial 39 finished with value: 0.1599097126556509 and parameters: {'n_estimators': 900, 'learning_rate': 0.05, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 29 with value: 0.15989894592325202.


[INFO] Final training took 167.77 seconds.
[INFO] Saving final Gradient Boosting model to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/models/Clip16_DreamSim_Lora_Default_123_Third/gradient_boosting_regressor_model.joblib and study to /content/drive/MyDrive/perceptual-vits-fashion-forecasting/studies/Clip16_DreamSim_Lora_Default_123_Third/gradient_boosting_regressor_study.joblib.


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/perceptual-vits-fashion-forecasting/models/Clip16_DreamSim_Lora_Default_123_Third/gradient_boosting_regressor_model.joblib'